getvalue RAX and simplify. before subsitution. Oh I already dio that... Hmm.

Maybe if I set init_RAX
Take the assert out of the smtlib statements. It's weird to have it in an assume
Versions that don't need conditions?

exit and entry are kind of subsumed. No but they aren't if I want 

If I ever jump to something marked as entry, don't do it. Just use entry condition, chaos, and receive exit condition. Or just run, but also receive exit condition?

Faster checks for true / false. DOn't bother doing a whole thing... right?


Mark out the pieces of memory that need to be loaded into memory for this to work.

kd_load foo: .ascii "hello world"
kd_load

should memstate have a pointer to a context? Maybe.

Also initialize all symbols that point to data to at least... eh.


mem ---> Lambda([x], mem(yada)[x])


- fixup to work on riscv and 32 bit.
- nicer error output. Negative examples
- Retain initial ghost variables. kd_ghost? kd_let ?
- Countermodels interpret to be more readable. Give path through program, state at beginning and end.
- Use abstractions so the intermidate symbolic states aren't quite so unreadable. Or initialize mem to RAX0. Too slow though. Hmm.
- Hoarize. I've had so many bugs this principled approach might be pretty useful.
- prearranged loaded data. Mem does not currently reflect the actual contents of the loading. kd_load ?
- pre and post conditions on calls? if jump into address already.
- objcopy assertions into section
- Pro / Con of WP?
- kd_always for constantly checked invariants
- A whitelist of jump points. Only allow labelled positions?
- Regular execution or GDB sessions. All the cbat features.
- failsafe. If I grep kd_ in the line, but don't recognize it, I should complain. Multiline smtlib would be nice too.
- The stdlib of helpers.
- speed
- Tracking seen addresses. We may want to track if we never hit a label that has an annotation on it, as this is suspicious.
- it could make sense to slot in AFL or angr or Symqemu or whatever instead of my system, checking the same properties. I have designed my semantics to be moderately sound
- ram8;    ram32 = Lambda([addr], selectconcat32(ram8, addr))   ram32 ram64
- comparative checking between two assembly programs


done:
- memory. Gotta inject a `mem` variable.
- Gotta test that stuff I put in the library
- cuts. Invariants are cuts of cfg into a dag. Distinguishing backedges from forward edges
- We should probably do asserts, assumes, assigns in program text order if multiple at single address. 
- store history just to make debugging easier.


cbmc / ebmc options to mimic? seelct entry points to test. select asssertions to test
bounded mode'


flag for eager VC or timeout param.

Grab just variables in highlevel assertion to present countermodel.

Hmm it would actually be easy to add a kd_label and let it persist through the parsing loop... Or just parse labels. There _has_ to have been a label in the contiguous previous lines




In [17]:
%%file /tmp/knuckle.s

#precondition
.macro kd_entry label smt_expr
\label :
.endm

.macro kd_assert label smt_expr
\label :
.endm

.macro kd_assume label smt_expr
\label :
.endm

#postcondition
.macro kd_exit label smt_expr 
\label :
.endm

#invariant
.macro kd_cut label smt_expr
\label :
.endm 

.macro kd_always smt_expr
.endm


Overwriting /tmp/knuckle.s


In [ ]:
%%file /tmp/stack.s
.include "/tmp/knuckle.s"
.global  _start
kd_entry _start " true"
    movq     $42, (%rsp)
kd_exit _start_end "(= (select ram RSP) (_ bv42 8))"
    ret

Overwriting /tmp/stack.s


In [1]:
from kdrag.all import *
import kdrag.contrib.pcode as pcode
from kdrag.contrib.pcode.asmspec import assemble_and_check
assemble_and_check("/tmp/stack.s")


Z3Exception: b'(error "line 2 column 12: unknown constant assert (Bool) ")\n'

# cut

cut, we maybe want post and pre program point
pre_insn1
exec_insn1
post_insn1
pre_insn2
...

examples:
VST book https://softwarefoundations.cis.upenn.edu/vc-current/toc.html
dafny book
frama-c examples / book https://link.springer.com/book/10.1007/978-3-031-55608-1 

sum-array
hmm. I'm worried that we'll need auxiliary predicates.
I could define them in assembly?

kd_prelude  "(define-fun foo ( )  )"
kd_declare  "(declare-const )"   # kdprelude "(declare-const mytemp (BitVec 64))"
kd_assign "mytemp" expr
kd_declare

"(define-const flub (RAX))"  Hmm. This might auto expand. I think so. So you can have a running summary in thisa way
But kd_assign gives you 

Maybe i should be worried more about the embedded python experience. Getting lemmas in there.
lemmas = dict[label, list[kd.Proof]]  Offer needed extra bits.

Or have the thing output a pile of verification conditions in Results.

This is getting verbose. I'm just delaying figuring out what I want to do. But it is also flexibility.

class Trace()
    events : list[Event]
    def VCs(self):

class TraceForest():

class Jump():
    TraceForest

class Event():
    pass
class Execute(Event): ...
class Assert(Event): ...
class Assume(Event):
class Entry():
class 

T





In [2]:
%%file /tmp/cut.s
.include "/tmp/knuckle.s"
.global  _start
kd_entry _start "true"
    movq     $42, %rdi
kd_cut mycut "(= RDI (_ bv42 64))"
    movq     %rdi, %rax
kd_exit _start_end "(= RAX (_ bv42 64))"
    ret

Overwriting /tmp/cut.s


In [2]:
from kdrag.all import *
import kdrag.contrib.pcode as pcode
from kdrag.contrib.pcode.asmspec import assemble_and_check
assemble_and_check("/tmp/cut.s")

{
  "entries": {
    "4194304": [
      [
        "_start",
        "true"
      ]
    ]
  },
  "asserts": {},
  "assumes": {},
  "exits": {
    "4194314": [
      [
        "_start_end",
        "(= RAX #x000000000000002a)"
      ]
    ]
  },
  "cuts": {
    "4194311": [
      [
        "mycut",
        "(= RDI #x000000000000002a)"
      ]
    ]
  }
}
entry _start at 4194304 with precond True
cut mycut at 4194311 with precond RDI == 42
Executing 0x400007/3: MOV RAX,RDI at (4194311, 0) PCODE IMARK ram[400007:3]
Executing 0x400007/3: MOV RAX,RDI at (4194311, 1) PCODE RAX = RDI
[✅] exit _start_end: RAX == 42
Executing 0x400000/7: MOV RDI,0x2a at (4194304, 0) PCODE IMARK ram[400000:7]
Executing 0x400000/7: MOV RDI,0x2a at (4194304, 1) PCODE RDI = 0x2a
[✅] cut mycut: RDI == 42


Results(successes=['[✅] exit _start_end: RAX == 42', '[✅] cut mycut: RDI == 42'], failures=[], traces=[[(4194311, 0), '[✅] exit _start_end: RAX == 42'], [(4194304, 0), '[✅] cut mycut: RDI == 42']])

In [3]:
%%file /tmp/cutloop.s
.include "/tmp/knuckle.s"
.global  _start
kd_entry _start "true"
    movq     $42, %rdi
kd_cut mycut "(= RDI (_ bv42 64))"
    jmp mycut

Writing /tmp/cutloop.s


In [1]:
from kdrag.all import *
import kdrag.contrib.pcode as pcode
from kdrag.contrib.pcode.asmspec import assemble_and_check
assemble_and_check("/tmp/cutloop.s")

{
  "entries": {
    "4194304": [
      [
        "_start",
        "true"
      ]
    ]
  },
  "asserts": {},
  "assumes": {},
  "exits": {},
  "cuts": {
    "4194311": [
      [
        "mycut",
        "(= RDI #x000000000000002a)"
      ]
    ]
  }
}
entry _start at 4194304 with precond True
cut mycut at 4194311 with invariant RDI == 42
Executing 0x400007/2: JMP 0x400007 at (4194311, 0) PCODE IMARK ram[400007:2]
Executing 0x400007/2: JMP 0x400007 at (4194311, 1) PCODE goto ram[400007:8]
[✅] cut mycut: RDI == 42
Executing 0x400000/7: MOV RDI,0x2a at (4194304, 0) PCODE IMARK ram[400000:7]
Executing 0x400000/7: MOV RDI,0x2a at (4194304, 1) PCODE RDI = 0x2a
[✅] cut mycut: RDI == 42


Results(successes=['[✅] cut mycut: RDI == 42', '[✅] cut mycut: RDI == 42'], failures=[], traces=[[(4194311, 0), '[✅] cut mycut: RDI == 42'], [(4194304, 0), '[✅] cut mycut: RDI == 42']])

In [ ]:
%%file /tmp/cutloop.s
.include "/tmp/knuckle.s"
.global  _start
kd_entry _start "true"
    movq     $42, %rdi
    lea      1(%rdi), %rax
kd_cut mycut "(bvule RDI (_ bv42 64))"
    add     $1, %rax
    jmp mycut


# riscv


In [6]:
%%file /tmp/mop42.s
.include "/tmp/knuckle.s"
    .text
    .globl  myfunc
kd_entry myfunc "(assert true)"
    li    a0, 42
kd_exit myfunc_end "(assert (= a0 (_ bv42 64)))"
    ret

Overwriting /tmp/mop42.s


In [24]:
! riscv64-linux-gnu-gcc -c -o /tmp/mop42.o /tmp/mop42.s

In [25]:
%%file /tmp/test.c
#include <stdio.h>
#include <stdint.h>
uint64_t myfunc();
int main() {
    printf("Result is %lu\n", myfunc());
    return 0;
}

Overwriting /tmp/test.c


In [26]:
! riscv64-linux-gnu-gcc /tmp/test.c /tmp/mop42.o -o /tmp/test && qemu-riscv64 -L /usr/riscv64-linux-gnu /tmp/test

Result is 42


In [5]:
%load_ext jupyterflame

In [2]:
#%%prun -s tottime
import subprocess
import kdrag.contrib.pcode as pcode
from kdrag.contrib.pcode.asmspec import AsmSpec, run_all_paths
def assemble_and_check_riscv64(filename: str):
    subprocess.run(["riscv64-linux-gnu-as", filename, "-o", "/tmp/kdrag_temp.o"], check=True)
    ctx = pcode.BinaryContext("/tmp/kdrag_temp.o", langid="RISCV:LE:64:default")
    spec = AsmSpec.of_file(filename, ctx)
    print(spec)
    return run_all_paths(ctx, spec)
assemble_and_check_riscv64("/tmp/mop42.s")

{
  "entries": {
    "4194304": [
      [
        "myfunc",
        "true"
      ]
    ]
  },
  "asserts": {},
  "assumes": {},
  "exits": {
    "4194308": [
      [
        "myfunc_end",
        "(= a0 #x000000000000002a)"
      ]
    ]
  },
  "cuts": {}
}
entry myfunc at 4194304 with precond True
Executing 0x400000/4: li a0,0x2a at (4194304, 0) PCODE IMARK ram[400000:4]
Executing 0x400000/4: li a0,0x2a at (4194304, 1) PCODE unique[780:8] = 0x2a
Executing 0x400000/4: li a0,0x2a at (4194304, 2) PCODE a0 = unique[780:8]
[✅] exit myfunc_end: a0 == 42


Results(successes=['[✅] exit myfunc_end: a0 == 42'], failures=[], traces=[[(4194304, 0), '[✅] exit myfunc_end: a0 == 42']])

# 32 bit
little endian and big endian




# Bits and Bobbles
https://github.com/WestfW/structured_gas



# reorg

    checks: defaultdict[int, list[HasCheck]] = dataclasses.field(
        default_factory=lambda: defaultdict(list)
    )


In [1]:
# %%prun -s tottime
from kdrag.contrib.pcode.asmspec import AsmSpec, run_all_paths, assemble_and_check_str

ex = """
    .include "/tmp/knuckle.s"
    .globl  _start
    kd_entry _start "true"
            movq     %rdi, %rax
            cmp     %rdi, %rsi
            cmovb   %rsi, %rax
    kd_exit _start_end "(= RAX (ite (bvult RDI RSI) RDI RSI))"
    #kd_exit _start_end "(or (= RAX RDI) (= RAX RSI))"
            ret
"""
assemble_and_check_str(ex)



{
  "addrmap": {
    "4194304": [
      {
        "label": "_start",
        "addr": 4194304,
        "expr": "true"
      }
    ],
    "4194314": [
      {
        "label": "_start_end",
        "addr": 4194314,
        "expr": "(= RAX (ite (bvult RDI RSI) RDI RSI))"
      }
    ]
  },
  "entries": {
    "4194304": [
      [
        "_start",
        "true"
      ]
    ]
  },
  "asserts": {},
  "assumes": {},
  "exits": {
    "4194314": [
      [
        "_start_end",
        "(= RAX (ite (bvult RDI RSI) RDI RSI))"
      ]
    ]
  },
  "cuts": {}
}
Executing 0x400000/3: MOV RAX,RDI at (4194304, 0) PCODE IMARK ram[400000:3]
Executing 0x400000/3: MOV RAX,RDI at (4194304, 1) PCODE RAX = RDI
Executing 0x400003/3: CMP RSI,RDI at (4194307, 0) PCODE IMARK ram[400003:3]
Executing 0x400003/3: CMP RSI,RDI at (4194307, 1) PCODE unique[3af00:8] = RSI
Executing 0x400003/3: CMP RSI,RDI at (4194307, 2) PCODE CF = unique[3af00:8] < RDI
Executing 0x400003/3: CMP RSI,RDI at (4194307, 3) PCODE OF = sbor

Results(successes=["[✅] VC(Entry(label='_start', addr=4194304, expr=True), ['0x400000', '0x400003', '0x400006'], Exit(label='_start_end', addr=4194314, expr=RAX == If(ULT(RDI, RSI), RDI, RSI)))", "[✅] VC(Entry(label='_start', addr=4194304, expr=True), ['0x400000', '0x400003', '0x400006'], Exit(label='_start_end', addr=4194314, expr=RAX == If(ULT(RDI, RSI), RDI, RSI)))"], failures=[], traces=[])

In [ ]:
import pypcode
ctx = pypcode.Context("RISCV:LE:64:default")
#ctx.registers
# Trim mem_init to just the good stuff somehow?
# Maybe in archinfo
{name: vnode for name, vnode in ctx.registers.items() if (vnode.size == 8 or vnode.size == 4) and len(name) <= 3}

{'pc': <pypcode.pypcode_native.Varnode at 0x76b63422a550>,
 'ra': <pypcode.pypcode_native.Varnode at 0x76b60ba68360>,
 'sp': <pypcode.pypcode_native.Varnode at 0x76b60ba68810>,
 'gp': <pypcode.pypcode_native.Varnode at 0x76b60ba685a0>,
 'tp': <pypcode.pypcode_native.Varnode at 0x76b60ba68480>,
 't0': <pypcode.pypcode_native.Varnode at 0x76b60ba68c30>,
 't1': <pypcode.pypcode_native.Varnode at 0x76b60ba68e40>,
 't2': <pypcode.pypcode_native.Varnode at 0x76b60ba68c00>,
 's0': <pypcode.pypcode_native.Varnode at 0x76b60ba68cc0>,
 's1': <pypcode.pypcode_native.Varnode at 0x76b60ba68d50>,
 'a0': <pypcode.pypcode_native.Varnode at 0x76b60ba68d20>,
 'a1': <pypcode.pypcode_native.Varnode at 0x76b60ba68d80>,
 'a2': <pypcode.pypcode_native.Varnode at 0x76b60ba68ed0>,
 'a3': <pypcode.pypcode_native.Varnode at 0x76b60ba69200>,
 'a4': <pypcode.pypcode_native.Varnode at 0x76b60ba68c90>,
 'a5': <pypcode.pypcode_native.Varnode at 0x76b60ba68f90>,
 'a6': <pypcode.pypcode_native.Varnode at 0x76b60ba68f00

In [3]:
from kdrag.all import *
import kdrag.contrib.pcode as pcode

ctx = pcode.BinaryContext()
memstate = ctx.init_mem()
ctx.get_reg(memstate, "RAX")



RAX!1506

In [ ]:
! sudo apt install gcc-riscv64-linux-gnu g++-riscv64-linux-gnu  binutils-riscv64-linux-gnu

In [16]:
import cle
loader = cle.loader.Loader("/tmp/myfunc.o")
dir(loader.all_elf_objects[0].arch)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_configure_capstone',
 '_configure_keystone',
 '_cs',
 '_cs_x86_syntax',
 '_get_register_dict',
 '_ks',
 '_ks_x86_syntax',
 'address_types',
 'argument_register_positions',
 'argument_registers',
 'artificial_registers',
 'artificial_registers_offsets',
 'asm',
 'bits',
 'bp_offset',
 'branch_delay_slot',
 'byte_width',
 'bytes',
 'cache_irsb',
 'call_pushes_ret',
 'call_sp_fix',
 'capstone',
 'capstone_support',
 'capstone_x86_syntax',
 'concretize_unique_registers',
 'copy',
 'cpu_flag_register_offsets_and_bitmasks_map',
 'cs_arch',
 'cs_mode',
 'default_endness',
 